# 最长回文子串

思路：
- 暴力法
    - 枚举所有长度大于2的子串，并依次它们判断是否为回文
- 中心扩散法
    - 回文左右成镜像，因此可以从中心向两边展开进行判断回文
    - 对每个中心进行回文判断，最后选取最长的回文 
    - 回文中心可能是一个字符或者两个字符，因此中心个数为2n-1   eg：abba 两个字符 abcba 一个字符 与回文奇偶有关系
- 动态规划
    - 根据一个回文去掉两头以后，剩下的部分依然是回文，构成状态转移
    - 状态转移方程 `dp[i][j] = (s[i] == s[j]) and dp[i + 1][j - 1]`
    - 边界条件是：表达式 [i + 1, j - 1] 不构成区间，即长度严格小于 2，即 j - 1 - (i + 1) + 1 < 2 ，整理得 j - i < 3
    - 这个结论很显然：当子串 s[i, j] 的长度等于 2 或者等于 3 的时候，我其实只需要判断一下头尾两个字符是否相等就可以直接下结论了。

        - 如果子串 s[i + 1, j - 1] 只有 1 个字符，即去掉两头，剩下中间部分只有 11 个字符，当然是回文；
        - 如果子串 s[i + 1, j - 1] 为空串，那么子串 s[i, j] 一定是回文子串。
        
      因此，在 s[i] == s[j] 成立和 j - i < 3 的前提下，直接可以下结论，dp[i][j] = true，否则才执行状态转移。





注意：
- 要对空字符串，一个字符特例进行判断
- 可以对字符串插入#字符，使他变为始终为奇数的字符串，再使用中心扩散进行计算，得出答案后去掉#

自己第一个思路和中心扩展法类似，但由于是从两边向里面推进，所以对一些特殊的回文如abb无法判断。

只能判断对称的回文

In [10]:
class Solution(object):
    def longestPalindrome(s):
        """
        :type s: str
        :rtype: str
        """
        s_len = len(s)
        flag = False
        cur = -1
        for i in range(s_len // 2):
            if not flag and s[i] == s[s_len-i-1]:
                flag = True
                cur = i
            if s[i] != s[s_len-i-1]:
                flag = False
                cur = -1
        if s_len == 0:
            return ""
        if cur == -1:
            return s[0]
        else:
            return  s[cur:s_len-cur]




if __name__ == '__main__':

    s = 'abb'
    print(Solution.longestPalindrome(s))

a


# 答案

## 中心扩展法

根据别人的修改的，for i in range(s_len-1): 如果len不减一的话，对于ac输入，输出c，但减不减一都能通过，说明力扣对2个答案都认可

个人更倾向减一符合逻辑，不会存在下标溢出，这个未报错的原因是 right < len(s)先判断了，所以未判断后面一个

In [4]:
class Solution(object):
    def longestPalindrome(self, s):
        """
        :type s: str
        :rtype: str
        """
        s_len = len(s)

        if s_len < 2:
            return s
        start = 0
        end = 0
        for i in range(s_len-1):
            len1 = Solution().expandAroundCenter(s,i,i)
            len2 = Solution().expandAroundCenter(s,i,i+1)
            max_len = max(len1,len2)
            if max_len > end - start:
                start = i - (max_len -1) // 2  #长度减一才能匹配2个字符为中心的回文，具体原理还不清楚
                end = i + max_len // 2
        return s[start:end+1]

    def expandAroundCenter(self,s,left,right):

        while left>=0 and right < len(s) and s[left] == s[right]:
            left = left - 1
            right = right + 1
        return right - left -1 #减1才是字符串长度

## 动态规划

In [1]:
class Solution(object):
    def longestPalindrome(self, s):
        """
        :type s: str
        :rtype: str
        """
        s_len = len(s)

        if s_len < 2:
            return s

        dp = [[False for _ in range(s_len)]for _ in range(s_len)]
        max_len = 1
        start = 0  #这两个参数主要处理ac等这种非回文字符串的情况，输出第一个字符，因为它无法进入dp判断，相当于默认第一个字符为最长回文

        for i in range(s_len):
            dp[i][i] = True

        for j in range(1,s_len ):
            for i in range(0,j):
                if s[i] == s[j]:
                    if j - i < 3:
                        dp[i][j] = True
                    else:
                        dp[i][j] = dp[i+1][j-1]
                else:
                    dp[i][j] = False

                if dp[i][j]:
                    cur_len = j - i + 1
                    if cur_len > max_len:
                        max_len = cur_len
                        start = i
        return s[start:start+max_len]